<a href="https://colab.research.google.com/github/crisbpadilla/DataScience-M2M-course-Datatalent/blob/main/Widget_withBokeh_excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import cividis
from bokeh.models import CrosshairTool

import pandas as pd

output_notebook()

columns=['year','state','candidate','party','candidatevotes','totalvotes']
df = pd.read_csv('1976-2016-president.csv',usecols=columns,dtype={'year':str})
state_group= df.groupby('state')
year=df['year'].unique()

color=cividis(len(state_group))
p=figure(tooltips= '$name @x:@y%',x_range= year)

state_line={}

for i,states in enumerate(state_group.groups):

  data=state_group.get_group(states)
  party_data=data[data['party']=='democrat']
  percentage=party_data['candidatevotes']/party_data['totalvotes']*100
  line= p.line(x=party_data['year'],y=percentage, color=color[i], name= states)

  state_line[states]=line

p.add_tools(CrosshairTool())
show(p)

In [17]:
for state_line in state_line.values():
  state_line.visible= False

show(p)

In [19]:
from ipywidgets import interact,SelectMultiple,IntRangeSlider

In [46]:
from bokeh.io import output_notebook, push_notebook


handle = show(p, notebook_handle=True)

select_line= SelectMultiple( options=state_line.keys(),
                            rows=10, description='Select state(s):')

@interact(states=select_line)
def update(states):
  for state_line_name,state_line_fig in state_line.items():
    if state_line_name in states:
      state_line_fig.visible= True
    else:
      state_line_fig.visible= False
  push_notebook()

interactive(children=(SelectMultiple(description='Select state(s):', options=('Alabama', 'Alaska', 'Arizona', …

In [44]:
print(state_group.groups.keys())


dict_keys(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'])
